In [19]:
!pip install rdkit-pypi
from pandas.plotting import table
from rdkit.Chem import Draw
from rdkit import Chem
from sklearn.metrics import r2_score
from torch_geometric.data import DataLoader
from torch_geometric.datasets import MoleculeNet
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from torch_geometric.utils import to_networkx
from torch.nn import Linear
  
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import pandas as pd
import pubchempy
import rdkit
import time
import torch
import torch.nn.functional as F 
import warnings
warnings.filterwarnings("ignore")

In [16]:
!pip install torch torch-xla
import torch
import torch_xla
import torch_xla.core.xla_model as xm

# Set the TPU device
device = xm.xla_device()

print("Using device:", device)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 MB 18.9 MB/s eta 0:00:00:00:0100:01


ImportError: /usr/local/lib/python3.10/dist-packages/_XLAC.cpython-310-x86_64-linux-gnu.so: undefined symbol: _ZN2at6native12cpu_fallbackERKN3c1014OperatorHandleEPSt6vectorINS1_6IValueESaIS6_EEbNS1_11DispatchKeyE

In [12]:
!pip install rdkit-pypi
!pip install pubchempy
!pip install torch-geometric
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv \
  -f https://data.pyg.org/whl/torch-$(python -c "import torch; print(torch.__version__)").html



Looking in links: https://data.pyg.org/whl/torch-2.4.1+cu121.html


In [22]:
dataset = MoleculeNet(root=".", name="lipo")
data = dataset[0]

In [ ]:
# embedding_size = 64
# class GCN(torch.nn.Module):
#     def __init__(self):
#         # Init parent
#         super(GCN, self).__init__()
#         torch.manual_seed(42)

#         # GCN layers
#         self.initial_conv = GCNConv(data.num_features, embedding_size)
#         self.conv1 = GCNConv(embedding_size, embedding_size)
#         self.conv2 = GCNConv(embedding_size, embedding_size)
#         self.conv3 = GCNConv(embedding_size, embedding_size)

#         # Output layer
#         self.out = Linear(embedding_size*2, 1)

#     def forward(self, x, edge_index, batch_index):
#         # First Conv layer
#         hidden = self.initial_conv(x, edge_index)
#         hidden = F.tanh(hidden)

#         # Conv layers
#         hidden = self.conv1(hidden, edge_index)
#         hidden = F.tanh(hidden)
#         hidden = self.conv2(hidden, edge_index)
#         hidden = F.tanh(hidden)
#         hidden = self.conv3(hidden, edge_index)
#         hidden = F.tanh(hidden)
          
#         # Global Pooling (stack different aggregations)
#         hidden = torch.cat([gmp(hidden, batch_index), 
#                             gap(hidden, batch_index)], dim=1)

#         # Classifier (Linear).
#         out = self.out(hidden)

#         return out, hidden

# model = GCN()
# print(model)
# print("Number of parameters: ", sum(p.numel() for p in model.parameters()))

In [ ]:
# from torch_geometric.data import DataLoader
# import warnings
# warnings.filterwarnings("ignore")

# # Root mean squared error
# loss_fn = torch.nn.MSELoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.000)  

# # Calculate accuracy r2
# def r2_accuracy(pred_y, y):
#     score = r2_score(y, pred_y)
#     return round(score, 2)*100

# # Data generated
# embeddings = []
# losses = []
# accuracies = []
# outputs = []
# targets = []

# # Use GPU for training, if available
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model = model.to(device)

# # Data loader
# data_size = len(dataset)
# NUM_GRAPHS_PER_BATCH = 64
# NUM_EPOCHS = 2000

# torch.manual_seed(12345)

# #randomize and split the data
# dataset = dataset.shuffle()

# train_dataset = dataset[:int(data_size * 0.8)]
# test_dataset = dataset[int(data_size * 0.8):]

# loader = DataLoader(train_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False)

# print('\n======== data distribution =======\n')
# print("Size of training data: {} graphs".format(len(train_dataset)))
# print("Size of testing data: {} graphs".format(len(test_dataset)))
 
# def train(data):
#     # Enumerate over the data
#     for batch in loader:
#       # Use GPU
#       batch.to(device)  
#       # Reset gradients
#       optimizer.zero_grad() 
#       # Passing the node features and the connection info
#       pred, embedding = model(batch.x.float(), batch.edge_index, batch.batch) 
#       # Calculating the loss and gradients
#       loss = loss_fn(pred, batch.y)     
#       acc = r2_accuracy(pred.detach().numpy(), batch.y.detach().numpy())

#       loss.backward()  
#       # Update using the gradients
#       optimizer.step()   
#     return loss, acc, pred, batch.y, embedding

# print('\n======== Starting training ... =======\n')
# start_time = time.time()

# losses = []
# for epoch in range(NUM_EPOCHS):
#     loss, acc, pred, target, h = train(data)
#     losses.append(loss)
#     accuracies.append(acc)
#     outputs.append(pred)
#     targets.append(target)
    
    
#   # print(f"Epoch {epoch} | Train Loss {loss}")
#     print(f'Epoch {epoch:>3} | Loss: {loss:.5f} | Acc: {acc:.2f}%')

# print("\nTraining done!\n")
# elapsed = time.time() - start_time
# minutes_e = elapsed//60
# print("--- training took:  %s minutes ---" % (minutes_e))

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv, global_mean_pool
from torch_geometric.data import DataLoader
from torch_geometric.datasets import MoleculeNet
from torch_geometric.transforms import NormalizeFeatures
from sklearn.metrics import r2_score

# Load the MoleculeNet dataset
# dataset = MoleculeNet(root="MoleculeNet", name="Lipophilicity", transform=NormalizeFeatures())

# Split dataset into training, validation, and test sets
torch.manual_seed(42)
dataset = dataset.shuffle()

# Ensure the node features are of type float
dataset.x = dataset.x.float()

train_size = int(0.8 * len(dataset))
val_size = int(0.2 * len(dataset))
# test_size = len(dataset) - train_size - val_size

train_dataset = dataset[:train_size]
val_dataset = dataset[train_size:train_size + val_size]
# test_dataset = dataset[train_size + val_size:]

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
# test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# print(f"Training molecules: {len(train_dataset)}, Validation: {len(val_dataset)}, Test: {len(test_dataset)}")

# Define the GraphSAGE Model
class GraphSAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GraphSAGE, self).__init__()
        
        # Define six GraphSAGE layers
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.conv3 = SAGEConv(hidden_channels, hidden_channels)
        self.conv4 = SAGEConv(hidden_channels, hidden_channels)
        self.conv5 = SAGEConv(hidden_channels, hidden_channels)
        self.conv6 = SAGEConv(hidden_channels, hidden_channels)
        self.conv7 = SAGEConv(hidden_channels, hidden_channels)
        self.conv8 = SAGEConv(hidden_channels, hidden_channels)
        self.conv9 = SAGEConv(hidden_channels, hidden_channels)
        self.conv10 = SAGEConv(hidden_channels, hidden_channels)

        # Linear layer to align input dimensions for skip connections
        self.project = torch.nn.Linear(in_channels, hidden_channels)

        # Fully connected layers
        self.fc1 = torch.nn.Linear(hidden_channels, hidden_channels)
        self.fc2 = torch.nn.Linear(hidden_channels, out_channels)

    def forward(self, data):
        # Unpack the data object
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = x.to(torch.float32)

        # Project input features to match hidden dimensions
        x_proj = self.project(x)

        # Apply SAGEConv layers with skip connections
        x1 = F.relu(self.conv1(x, edge_index))
        x2 = F.relu(self.conv2(x1 + x_proj, edge_index))  # Skip connection from input to second layer
        x3 = F.relu(self.conv3(x2 + x1, edge_index))      # Skip connection from x1 to x3
        x4 = F.relu(self.conv4(x3 + x2, edge_index))      # Skip connection from x2 to x4
        x5 = F.relu(self.conv5(x4 + x3, edge_index))      # Skip connection from x3 to x5
        x6 = F.relu(self.conv6(x5 + x4, edge_index))  
        # x7 = F.relu(self.conv7(x6 + x5, edge_index))      
        # x8 = F.relu(self.conv8(x7 + x6, edge_index))
        # x9 = F.relu(self.conv7(x8 + x7, edge_index))      
        # x10 = F.relu(self.conv8(x9 + x8, edge_index))
        # Pool node embeddings into graph embeddings (mean pooling)
        x = global_mean_pool(x6, batch)

        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x
# Instantiate the Model, Optimizer, and Loss Function
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model = GraphSAGE(
    in_channels=9,  # Input size matches node feature size
    hidden_channels=128,  # Hidden layer size
    out_channels=1  # Output size (1 for regression task)
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
criterion = torch.nn.MSELoss()

# Define the Training Loop
def train(model, loader):
    model.train()
    total_loss = 0
    all_true = []
    all_pred = []
    for data in loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
        # Collect true and predicted values for R² score calculation
        all_true.append(data.y.cpu().numpy())
        all_pred.append(out.cpu().detach().numpy())
    
    # Flatten the lists and compute R² score
    all_true = np.concatenate(all_true)
    all_pred = np.concatenate(all_pred)
    r2 = r2_score(all_true, all_pred)
    
    return total_loss / len(loader), r2

# Define the Validation and Testing Loop
@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    total_loss = 0
    all_true = []
    all_pred = []
    for data in loader:
        data = data.to(device)
        out = model(data)
        loss = criterion(out, data.y)
        total_loss += loss.item()
        
        # Collect true and predicted values for R² score calculation
        all_true.append(data.y.cpu().numpy())
        all_pred.append(out.cpu().detach().numpy())
    
    # Flatten the lists and compute R² score
    all_true = np.concatenate(all_true)
    all_pred = np.concatenate(all_pred)
    r2 = r2_score(all_true, all_pred)
    
    return total_loss / len(loader), r2

# Train the Model
epochs = 2000
for epoch in range(1, epochs + 1):
    train_loss, train_r2 = train(model, train_loader)
    val_loss, val_r2 = evaluate(model, val_loader)
    print(f"Epoch: {epoch:03d}, Train Loss: {train_loss:.4f}, Train R²: {train_r2:.4f}, Validation Loss: {val_loss:.4f}, Validation R²: {val_r2:.4f}")

# Test the Model
test_loss, test_r2 = evaluate(model, val_loader)
print(f"Test Loss: {test_loss:.4f}, Test R²: {test_r2:.4f}")


cuda:0
Epoch: 001, Train Loss: 1.5845, Train R²: -0.0765, Validation Loss: 1.3106, Validation R²: 0.0210
Epoch: 002, Train Loss: 1.4874, Train R²: -0.0105, Validation Loss: 1.9283, Validation R²: -0.4274
Epoch: 003, Train Loss: 1.4931, Train R²: -0.0143, Validation Loss: 1.2751, Validation R²: 0.0469
Epoch: 004, Train Loss: 1.4899, Train R²: -0.0122, Validation Loss: 1.3047, Validation R²: 0.0274
Epoch: 005, Train Loss: 1.4947, Train R²: -0.0154, Validation Loss: 1.2696, Validation R²: 0.0509
Epoch: 006, Train Loss: 1.4221, Train R²: 0.0339, Validation Loss: 1.3586, Validation R²: -0.0101
Epoch: 007, Train Loss: 1.4435, Train R²: 0.0193, Validation Loss: 1.4588, Validation R²: -0.0828
Epoch: 008, Train Loss: 1.4142, Train R²: 0.0392, Validation Loss: 1.2889, Validation R²: 0.0401
Epoch: 009, Train Loss: 1.4229, Train R²: 0.0334, Validation Loss: 1.2607, Validation R²: 0.0591
Epoch: 010, Train Loss: 1.4138, Train R²: 0.0395, Validation Loss: 1.2389, Validation R²: 0.0736
Epoch: 011, Tra

In [29]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv, global_mean_pool
from torch_geometric.data import DataLoader
from sklearn.metrics import r2_score
import numpy as np

# Define the Graph Attention Network Model
class GATNet(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads=4):
        super(GATNet, self).__init__()
        
        # Define GAT layers with skip connections
        self.conv1 = GATConv(in_channels, hidden_channels, heads=heads, concat=True)
        self.conv2 = GATConv(hidden_channels * heads, hidden_channels, heads=heads, concat=True)
        self.conv3 = GATConv(hidden_channels * heads, hidden_channels, heads=heads, concat=True)
        self.conv4 = GATConv(hidden_channels * heads, hidden_channels, heads=heads, concat=True)
        self.conv5 = GATConv(hidden_channels * heads, hidden_channels, heads=heads, concat=True)

        # Final GAT layer outputting a single head for aggregation
        self.conv6 = GATConv(hidden_channels * heads, hidden_channels, heads=1, concat=False)

        # Fully connected layers
        self.fc1 = torch.nn.Linear(hidden_channels, hidden_channels)
        self.fc2 = torch.nn.Linear(hidden_channels, out_channels)

        # Project input dimensions for alignment with hidden layers
        self.project = torch.nn.Linear(in_channels, hidden_channels * heads)

    def forward(self, data):
        # Unpack the data object
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = x.to(torch.float32)

        # Project input features to hidden dimensions
        x_proj = self.project(x)

        # Apply GAT layers with skip connections
        x1 = F.relu(self.conv1(x, edge_index))
        x2 = F.relu(self.conv2(x1 + x_proj, edge_index))  # Skip connection from input to x2
        x3 = F.relu(self.conv3(x2 + x1, edge_index))      # Skip connection from x1 to x3
        x4 = F.relu(self.conv4(x3 + x2, edge_index))      # Skip connection from x2 to x4
        x5 = F.relu(self.conv5(x4 + x3, edge_index))      # Skip connection from x3 to x5
        x6 = self.conv6(x5 + x4, edge_index)              # Final GAT layer with a single head

        # Pool node embeddings into graph embeddings (mean pooling)
        x = global_mean_pool(x6, batch)

        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x

# Initialize the Model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = GATNet(
    in_channels=9,  # Input size matches node feature size
    hidden_channels=128,  # Hidden layer size
    out_channels=1,  # Output size (1 for regression task)
    heads=4  # Number of attention heads
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
criterion = torch.nn.MSELoss()

# Define the Training Loop
def train(model, loader):
    model.train()
    total_loss = 0
    all_true = []
    all_pred = []
    for data in loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
        # Collect true and predicted values for R² score calculation
        all_true.append(data.y.cpu().numpy())
        all_pred.append(out.cpu().detach().numpy())
    
    # Flatten the lists and compute R² score
    all_true = np.concatenate(all_true)
    all_pred = np.concatenate(all_pred)
    r2 = r2_score(all_true, all_pred)
    
    return total_loss / len(loader), r2

# Define the Validation Loop
@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    total_loss = 0
    all_true = []
    all_pred = []
    for data in loader:
        data = data.to(device)
        out = model(data)
        loss = criterion(out, data.y)
        total_loss += loss.item()
        
        # Collect true and predicted values for R² score calculation
        all_true.append(data.y.cpu().numpy())
        all_pred.append(out.cpu().detach().numpy())
    
    # Flatten the lists and compute R² score
    all_true = np.concatenate(all_true)
    all_pred = np.concatenate(all_pred)
    r2 = r2_score(all_true, all_pred)
    
    return total_loss / len(loader), r2

# Train the Model
epochs = 2000
for epoch in range(1, epochs + 1):
    train_loss, train_r2 = train(model, train_loader)
    val_loss, val_r2 = evaluate(model, val_loader)
    print(f"Epoch: {epoch:03d}, Train Loss: {train_loss:.4f}, Train R²: {train_r2:.4f}, Validation Loss: {val_loss:.4f}, Validation R²: {val_r2:.4f}")

# Test the Model
test_loss, test_r2 = evaluate(model, val_loader)
print(f"Test Loss: {test_loss:.4f}, Test R²: {test_r2:.4f}")


Epoch: 001, Train Loss: 2.0337, Train R²: -0.4006, Validation Loss: 1.4866, Validation R²: -0.0397
Epoch: 002, Train Loss: 1.4822, Train R²: -0.0208, Validation Loss: 1.3974, Validation R²: 0.0259
Epoch: 003, Train Loss: 1.4578, Train R²: -0.0040, Validation Loss: 1.3607, Validation R²: 0.0520
Epoch: 004, Train Loss: 1.4652, Train R²: -0.0091, Validation Loss: 1.4650, Validation R²: -0.0256
Epoch: 005, Train Loss: 1.4259, Train R²: 0.0180, Validation Loss: 1.7520, Validation R²: -0.2304
Epoch: 006, Train Loss: 1.4644, Train R²: -0.0086, Validation Loss: 1.2900, Validation R²: 0.1053
Epoch: 007, Train Loss: 1.2524, Train R²: 0.1374, Validation Loss: 1.2648, Validation R²: 0.1215
Epoch: 008, Train Loss: 1.2073, Train R²: 0.1685, Validation Loss: 1.2314, Validation R²: 0.1426
Epoch: 009, Train Loss: 1.1997, Train R²: 0.1737, Validation Loss: 1.0897, Validation R²: 0.2371
Epoch: 010, Train Loss: 1.1776, Train R²: 0.1890, Validation Loss: 1.1432, Validation R²: 0.1967
Epoch: 011, Train Loss